<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Первичный-анализ" data-toc-modified-id="Первичный-анализ-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Первичный анализ</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Разделение на выборки</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LightGBM</a></span></li></ul></li><li><span><a href="#Проверка-модели" data-toc-modified-id="Проверка-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Токсичные комментарии

Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Первичный анализ

In [18]:
import pandas as pd
import numpy as np
import nltk
import re
import lightgbm as lgb
from tqdm import tqdm
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 

In [2]:
try:
    data = pd.read_csv('toxic_comments.csv', index_col=0)
except:
    data = pd.read_csv("/datasets/toxic_comments.csv", index_col=0)

In [3]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [5]:
data.isna().sum()

text     0
toxic    0
dtype: int64

### Лемматизация

In [6]:
corpus = data['text'].values.astype('U')

In [7]:
lemmatizer = WordNetLemmatizer()

In [8]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,            
                "N": wordnet.NOUN,           
                "V": wordnet.VERB,           
                "R": wordnet.ADV             
               }  
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    text = text.lower()
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', text).split()
    lemm_text = " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in cleared_text])
    return lemm_text

In [9]:
try:
    data['lemm_text'] = pd.read_csv("clear_text").values.astype('U')
except:
    tqdm.pandas()
    data['lemm_text'] = data['text'].progress_apply(lemmatize_text)

In [10]:
data = data.drop(['text'], axis=1)

### Разделение на выборки


In [11]:
stopwords = set(nltk_stopwords.words('english'))

In [12]:
features = data['lemm_text']
target = data["toxic"]

In [13]:
features_train, features_valid1, target_train, target_valid1 = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)

In [14]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid1, target_valid1, test_size= 0.5, random_state = 12345, stratify= target_valid1)

In [15]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

tf_idf = count_tf_idf.fit_transform(features_train) 
print("Размер матрицы:", tf_idf.shape)

Размер матрицы: (95575, 111735)


## Обучение

### Логистическая регрессия

### LightGBM

In [20]:
%%time
best_result = 0.7
best_model_LGBM = None
best_est = 0
best_depth = 0

for n_est in range(51,101, 10):
    for depth in range(1,15):  
        model = lgb.LGBMClassifier(num_leaves=31, max_depth=depth, learning_rate=0.1, n_estimators=n_est, random_state=12345,class_weight='balanced')
        model.fit(tf_idf, target_train)
        predicted = model.predict(tf_idf_valid)
        result = f1_score(target_valid,predicted)
        
        if result > best_result:
            best_model_LGBM = model
            best_result = result
            best_est = n_est
            best_depth = depth
print("RMSE наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth)  

RMSE наилучшей модели на валидационной выборке: 0.7341507177033493 Количество деревьев: 91 Максимальная глубина: 12
CPU times: total: 2h 51min 31s
Wall time: 17min 24s


## Проверка модели

## Выводы

В связи с колоссальным ростом количества различных игр, приложений, сайтов, появилась необходимость в обработке массива сообщений, в данном случае задача состояла в поиске NLP модели, которая будет показывать является ли комментарий негативным или позитивным для магазина викишоп. Данные прошли обработку, проведена лемматизация, токенизация, перевод данных в в векторный вид, разделение всей выборки на обучающую валидационную и тестовую выборки. Затем при выборе из двух моделей: Логистической регрессии и Градиентным бустингом, лучшей оказалась Логистическая регрессия с показателем на валидационной по метрике f1_score = 0.76, а на тестовой 0.75. Условия поставленной задачи(показатель f1_score не больше 0.75) выполнены.